# **Basic But Dumb Algorithm**
Why is it dumb? Uhm.. I'll answer it later

In [34]:
import random
import pandas as pd

In [35]:
content_categories = ["cat", "dog", "space"]

In [36]:
columns = ["video_id", "content_category", "like", "score" ]

In [37]:
content = []

#lets generate some content, in real world this is generated by the users
for i in range(100):
    video_id = f"video_{i}"
    content_category = content_categories[random.randint(0,2)] #for now we assume we already know "what" the content it about.
    like = 0 #0 indicates it is not liked by user
    score = like #if liked, then score is updated
    content.append([video_id, content_category, like, score ])


In [38]:
df = pd.DataFrame(data=content, columns=columns)
df


,video_id,content_category,like,score
0,video_0,space,0,0
1,video_1,cat,0,0
2,video_2,space,0,0
3,video_3,dog,0,0
4,video_4,cat,0,0
...,...,...,...,...
95,video_95,cat,0,0
96,video_96,dog,0,0
97,video_97,space,0,0
98,video_98,space,0,0


In [ ]:
#now lets try to make a very rudimentary system to mimic the interaction
#if you create a new account on any social media it will show the most popular content curated by mods or will throw random content at you.
#and then it will learn based on how you interact. 
#for simplicity we are going to use random content strategy. 
logs =[] #to keep a track of user's preferences

while True:
    len_content = len(df)
    random_int = random.randint(0, len_content-1)
    
    display_content_dict = df.iloc[random_int].to_dict()
    if display_content_dict["like"] == 0:
        response = input(f"Current Video: {display_content_dict['video_id']} is about {display_content_dict['content_category']}.\n Press y to like, or click enter to see new content")
        if response == "y":
            df.at[random_int, "like"] = 1
            df.at[random_int, "score"] += 1
            print(f"You liked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        if response == "" or response == "n":
            df.at[random_int, "like"] = -1
            df.at[random_int, "score"] += -1
            print(f"You disliked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        elif response == "q":
            break

    if display_content_dict["like"] == 1 or display_content_dict["like"] == -1:
        if input(f"Press q to quit") == "q":
            break


You liked video_59 about dog
You liked video_66 about dog
You disliked video_96 about dog
You liked video_94 about cat
You liked video_57 about dog
You disliked video_92 about cat
You disliked video_15 about space
You disliked video_76 about space
You liked video_83 about cat
You liked video_31 about dog


In [40]:
logs_df = pd.DataFrame(data=logs)
logs_df

,video_id,content_category,like,score
0,video_59,dog,1,1
1,video_66,dog,1,1
2,video_96,dog,-1,-1
3,video_94,cat,1,1
4,video_57,dog,1,1
5,video_92,cat,-1,-1
6,video_15,space,-1,-1
7,video_76,space,-1,-1
8,video_83,cat,1,1
9,video_31,dog,1,1


In [41]:
logs_df.groupby("content_category").describe()

like                                              score  \
                 count      mean       std  min  25%  50%  75%  max count   
content_category                                                            
cat                3.0  0.333333  1.154701 -1.0  0.0  1.0  1.0  1.0   3.0   
dog                5.0  0.600000  0.894427 -1.0  1.0  1.0  1.0  1.0   5.0   
space              2.0 -1.000000  0.000000 -1.0 -1.0 -1.0 -1.0 -1.0   2.0   

                                                               
                      mean       std  min  25%  50%  75%  max  
content_category                                               
cat               0.333333  1.154701 -1.0  0.0  1.0  1.0  1.0  
dog               0.600000  0.894427 -1.0  1.0  1.0  1.0  1.0  
space            -1.000000  0.000000 -1.0 -1.0 -1.0 -1.0 -1.0

In [42]:
# Here you can see, the user interacted with 11 videos out of which 3 were cat videos, 7 were dog videos, and 2 were space related videos. 
# Lets take a look at mean of all to get an idea of what user feels about the content categories.
# The user likes the dog content most, followed by neutral opinions about space, and then at last cat. 
# what can we infer from this? user is interested in dog content most, then space and then cat.
# that means the algorithm should show more dog content, a bit of space related content and then cat every now and then because user did like one video right?

### Lets see the same thing at scale, with more topics/content_categories, and more interaction methods such as time spent, sharing to friends, commenting, etc.
# **Slightly Better Algorithm**